In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = '../data/'
data_out = '../results/data_out/'

In [ ]:
util = ['221000']
utils = ['221100', '221200', '221300']
elec_tech = ['221111', '221112', '221113', '221114', '221115', '221116', '221117', '221118']

In [ ]:
cols = ['naics', 'naics_title', 'tot_emp', 'a_mean', 'a_median', 'emp_prse']
elec_jobs = pd.DataFrame(columns=['naics', 'naics_title', 'tot_emp', 'a_mean', 'a_median'])

yr = '18'
folder = 'oesm' + yr + 'in4/oesm' + yr + 'in4'
suff = '.xlsx'
filename_end = ''

occ_naics_3 = pd.read_excel(data + 'BLS/' + folder + \
                                '/nat3d_M20' + yr + '_dl' + filename_end + suff)

occ_naics_3.columns = occ_naics_3.columns.str.lower()
occ_naics_3 = occ_naics_3[(occ_naics_3['occ_code'] == '00-0000') & (occ_naics_3['naics'].astype(str).isin(util))]

occ_naics_4 = pd.read_excel(data + 'BLS/' + folder + \
                            '/nat4d_M20' + yr + '_dl' + filename_end + suff)
occ_naics_4.columns = occ_naics_4.columns.str.lower()

occ_naics_4 = occ_naics_4[(occ_naics_4['occ_code'] == '00-0000') & (occ_naics_4['naics'].astype(str).isin(utils))]

occ_naics_56 = pd.read_excel(data + 'BLS/' + folder + \
                             '/nat5d_6d_M20' + yr + '_dl' + suff)

occ_naics_56.columns = occ_naics_56.columns.str.lower()
occ_naics_56 = occ_naics_56[(occ_naics_56['occ_code'] == '00-0000') & (occ_naics_56['naics'].astype(str).isin(elec_tech))]

elec_jobs = pd.concat([elec_jobs, \
                          occ_naics_3[['naics', 'naics_title', 'tot_emp', 'a_mean', 'a_median']], \
                          occ_naics_4[['naics', 'naics_title', 'tot_emp', 'a_mean', 'a_median']], \
                          occ_naics_56[['naics', 'naics_title', 'tot_emp', 'a_mean', 'a_median']]], ignore_index=True)

In [ ]:
elec_jobs['tot_wage'] = elec_jobs['tot_emp'] * elec_jobs['a_mean']
elec_jobs.naics = elec_jobs.naics.astype(int)
elec_jobs

In [ ]:
elec_jobs[elec_jobs.naics.astype(str).isin(['221111', '221112', '221113', '221114', '221115', '221116', '221117', '221118'])].sum()

In [ ]:
cols = ['naics', 'naics_title', 'occ_code', 'occ_title', 'occ_group', 'tot_emp', 'a_mean', 'a_median', 'emp_prse']

yr = '18'
folder = 'oesm' + yr + 'in4/oesm' + yr + 'in4'
suff = '.xlsx'
filename_end = ''

occ_naics_3 = pd.read_excel(data + 'BLS/' + folder + \
                                '/nat3d_M20' + yr + '_dl' + filename_end + suff)

occ_naics_3.columns = occ_naics_3.columns.str.lower()
occ_naics_3 = occ_naics_3[occ_naics_3['naics'].astype(str).isin(util)]

occ_naics_4 = pd.read_excel(data + 'BLS/' + folder + \
                            '/nat4d_M20' + yr + '_dl' + filename_end + suff)
occ_naics_4.columns = occ_naics_4.columns.str.lower()

occ_naics_4 = occ_naics_4[occ_naics_4['naics'].astype(str).isin(utils)]

occ_naics_56 = pd.read_excel(data + 'BLS/' + folder + \
                             '/nat5d_6d_M20' + yr + '_dl' + suff)

occ_naics_56.columns = occ_naics_56.columns.str.lower()
occ_naics_56 = occ_naics_56[occ_naics_56['naics'].astype(str).isin(elec_tech)]

elec_workers = pd.concat([occ_naics_3[cols], \
                          occ_naics_4[cols], \
                          occ_naics_56[cols]], ignore_index=True)

In [ ]:
elec_workers.naics = elec_workers.naics.astype(int)
elec_workers

### Crosswalk and add to datasets

In [ ]:
bls_ipums = pd.read_csv(data_out + 'xwalk_asec_bls_2011-2019.csv', index_col=0)

In [ ]:
# only pick those occupations in crosswalk
elec_workers_ipums = elec_workers[elec_workers['occ_code'].isin(bls_ipums.index)]

# remove duplicates
elec_workers_ipums = elec_workers_ipums[~((elec_workers_ipums[['naics', 'occ_code']].duplicated(keep=False)) & (elec_workers_ipums['occ_group'] == 'detailed'))]

elec_workers_ipums.naics = elec_workers_ipums.naics.astype(int)

#### Manual fixes. Some total employment data is not known ('*' in BLS data). We manually fix that

In [ ]:
manual = pd.read_csv(data_out + 'manual_tot_emp_utilities.csv')

In [ ]:
for i, row in manual.iterrows():
    elec_workers_ipums.loc[(elec_workers_ipums['naics'] == row['naics']) & (elec_workers_ipums['occ_code'] == row['occ-code']), 'tot_emp'] = row['tot_emp']
    elec_workers.loc[(elec_workers['naics'] == row['naics']) & (elec_workers['occ_code'] == row['occ-code']), 'tot_emp'] = row['tot_emp']

# fixes below are to the higher levels, to get the imputation algorithm to work below
elec_workers.loc[(elec_workers['naics'] == 221111) & (elec_workers['occ_code'] == '51-9000'), 'tot_emp'] = 210
elec_workers.loc[(elec_workers['naics'] == 221112) & (elec_workers['occ_code'] == '53-2000'), 'tot_emp'] = 0
elec_workers.loc[(elec_workers['naics'] == 221112) & (elec_workers['occ_code'] == '53-6000'), 'tot_emp'] = 60
elec_workers.loc[(elec_workers['naics'] == 221112) & (elec_workers['occ_code'] == '49-9040'), 'tot_emp'] = 3330
elec_workers.loc[(elec_workers['naics'] == 221115) & (elec_workers['occ_code'] == '49-9040'), 'tot_emp'] = 430
elec_workers.loc[(elec_workers['naics'] == 221115) & (elec_workers['occ_code'] == '51-1000'), 'tot_emp'] = 80
elec_workers.loc[(elec_workers['naics'] == 221118) & (elec_workers['occ_code'] == '13-1000'), 'tot_emp'] = 100

In [ ]:
elec_workers_ipums[elec_workers_ipums.tot_emp == '**']

In [ ]:
sum_of_workers = elec_workers_ipums[['naics', 'tot_emp']].groupby('naics').sum()
sum_of_workers.columns = ['sum_of_occ']
sum_of_workers['tot_emp'] = elec_jobs[['naics', 'tot_emp']].set_index('naics')

In [ ]:
#BLS's combined total is more than the sum of the parts due to small occupations dropping out of the sample
sum_of_workers['diff'] = sum_of_workers.tot_emp / sum_of_workers.sum_of_occ
sum_of_workers['diff abs'] = sum_of_workers.tot_emp - sum_of_workers.sum_of_occ
sum_of_workers

In [ ]:
# large industries we rescale to fit total employment
ncs = [221000, 221100, 221200, 221300]
for naic in ncs:
    elec_workers_ipums.loc[elec_workers_ipums.naics == naic, 'tot_emp'] *=  sum_of_workers.loc[naic, 'diff']

#### For the smaller industries there are two ways to fill to the total number of workers
1. Impute workers for new occuaptions
2. Rescale workers in existing employment to fit total

We take a three-step approach
1. We check if the 'minor' occupation has more workers, and if so, we add workers of all occuaptions in that minor category
2. We check if the 'major' occupation has more workers, and, if so, we rescale all minor workers to fit the major category
3. We do the same at the 'total' level for each industry

In [ ]:
occ_names = pd.read_csv(data_out + 'occ_names_bls_minor_major.csv', index_col=0)

In [ ]:
elec_workers_ipums.reset_index(inplace=True, drop=True)

In [ ]:
def occUp(occ, lvl):
    if lvl == 0:
            return occ[:-1] + 1 * '0', 1
    if lvl == 1:
            return occ[:-3] + 3 * '0', 3
    if lvl == 3:
            return occ[:-4] + 4 * '0', 4
    if lvl == 4:
            return '00-0000', 6
    if lvl == 6:
            return '00-0000', 6

def get_lvl(occ):
    lvl = 2*len(occ) - len(occ.lstrip('0')) - len(occ.rstrip('0'))
    if lvl == 0:
            return 'detailed'
    if lvl == 1:
            return 'broad'
    if lvl in [2,3]:
            return 'minor'
    if lvl == 4:
            return 'major'
    if lvl == 6:
            return 'total'
        
def get_occ_oth(occ, lvl, df, tech = None, occ_lab = 'occ_code'):
    if tech is not None:
        df = df.loc[(df['naics'] == tech), :].copy()
    occ_other = []
    if lvl == 0:
        return False
    elif lvl in [1,4]:
        for i in range(1, 10):
            occ_oth = occ[:-lvl] + str(i) + '0' * (lvl-1)
            if df.loc[df[occ_lab] == occ_oth].size > 0:\
                occ_other += [occ_oth]
    elif lvl == 3:
        for i in range(1, 100):
            if i < 10:
                occ_oth = occ[:-lvl] + '0'+  str(i) + '0'
            else:
                occ_oth = occ[:-lvl] + str(i) + '0'
            if df.loc[df[occ_lab] == occ_oth].size > 0:\
                occ_other += [occ_oth]
    elif lvl == 6:
        for i in range(10, 100):
            occ_oth = str(i) + '-0000'
            if df.loc[df[occ_lab] == occ_oth].size > 0:\
                occ_other += [occ_oth]
    return occ_other

def get_occ_oth_full(occ, lvl, df, tech=None, occ_lab='occ_code'):
    if tech is not None:
        df = df.loc[(df['naics'] == tech), :].copy()
    occ_other = []
    if lvl == 0:
        return False
    elif lvl in [1,3,4]:
        for i in range(1, 10**lvl):
            x = len(str(i))
            occ_oth = occ[:-lvl] + (lvl-x) * '0' + str(i)
            if df.loc[df[occ_lab] == occ_oth].size > 0:\
                occ_other += [occ_oth]
    elif lvl == 6:
        occ_other = list(df[occ_lab])

    return occ_other

def sort_by_lvl(lst):
    tot = [i for i in lst if i == '00-0000']
    maj = [i for i in lst if i[-4:] == '0000' and i !='00-0000']
    minor = [i for i in lst if i[-3:] == '000' and i[-4] != '0']
    brd = [i for i in lst if i[-2:] == '00' and i[-3] != '0']
    brd2 = [i for i in lst if i[-1] == '0' and i[-2] != '0']
    det = [i for i in lst if i[-1] != '0']

    return det + brd2 + brd + minor + maj + tot

def minor_no_child(tot, loc):
    ret = []
    for occ in set(tot) - set(loc):
        lvl = 2*len(occ) - len(occ.lstrip('0')) - len(occ.rstrip('0'))
        if lvl in [2,3]: # if minor occupation
            children = [x for x in loc if x[:4] == occ[:4]] # if first 4 digits same, share same minor level
            if len(children) == 0:
                ret.append(occ)
    return ret
            

In [ ]:
elec_workers_ipums['occ_up'] = None

In [ ]:
tiny_frac = 0.002 # remove occupations smaller than 0.2% of total industry employment
tiny_abs = 30     # remove occupations smaller than 30 workers
        
df = elec_workers_ipums.copy()
df_tot = elec_workers.copy()
df_full = occ_names.copy()

for tech in elec_tech: #221111-221118
    tech = int(tech)
    print('now imputing values for ' + str(tech))
    
    # every occupation on every level in BLS
    total_occ_set =  list(set(df_tot.loc[df_tot['naics'] == tech, 'occ_code']))
    
    # the occupations that are part of our set
    local_occ_set = list(set(df.loc[df['naics'] == tech, 'occ_code']))
    
    # We check if there are any minor occupations absent without broad/detailed occs in local set
    minor_set = minor_no_child(total_occ_set, local_occ_set)

    # We loop through all local occupations, adding higher levels as we go through
    while len(local_occ_set) > 0:
        minr = False
        local_occ_set = sort_by_lvl(local_occ_set) # get most detailed first
        if len(minor_set) > 0:
            occ = minor_set.pop(0)
            local_occ_set.insert(0, occ)
            minr = True
        occ = local_occ_set.pop(0)
        if occ == '00-0000':
            continue
        lvl = 2*len(occ) - len(occ.lstrip('0')) - len(occ.rstrip('0')) #0 is detailed, 1 broad, ..., 6 total
        if lvl == 2:
            print('minor at lvl 2 for ' + occ)
            lvl = 3
        if minr:
            occ_up = occ
        else: # not minr:
            occ_up, lvl = occUp(occ, lvl)
        occ_oth = get_occ_oth(occ_up, lvl, df_tot, tech = tech) # all occuaptions one level below
        occ_all = get_occ_oth_full(occ_up, lvl, df) # all occupations any level below
        sum_emp = df.loc[(df['naics'] == tech) & (df['occ_code'].isin(occ_all)), 
                         ['occ_code', 'tot_emp']].drop_duplicates()['tot_emp'].sum()
        occ_up_emp  = df_tot.loc[(df_tot['naics'] == tech) & (df_tot['occ_code'] == occ_up), 'tot_emp'].values
        occ_up_prse = df_tot.loc[(df_tot['naics'] == tech) & (df_tot['occ_code'] == occ_up), 'emp_prse'].values
        if len(occ_up_emp) > 0:
            occ_up_emp  = occ_up_emp[0]
            occ_up_prse = occ_up_prse[0]
        else:
            print('no value found for upper level occ ' + occ_up)
            local_occ_set = [i for i in local_occ_set if i not in occ_oth]
            continue
            
        # diff is the number of jobs that are unaccounted for in that occupational category
        diff = occ_up_emp - sum_emp
        
        # for broad and minor level, we add occupations
        # for major and total, we scale up those we already have
        if (diff > 25) and (lvl in [1,3]):
                # add occupations, any that is part of our dataset
                new_occ_all = get_occ_oth_full(occ_up, lvl, df_full, occ_lab = 'OCC_CODE') # all possible occupations in our set
                new_occ = [item for item in new_occ_all if item not in occ_oth] # don't add occupations we already have
                sum_emp_full = df_full.loc[df_full['OCC_CODE'].isin(new_occ), 'TOT_EMP'].sum()
                for o in new_occ:
                    e = df_full.loc[df_full['OCC_CODE'] == o, 'TOT_EMP'].sum()
                    if e > 0:
                        # for each imputed occupation, we add proportional to the total employment in that
                        # occupational category economy-wide
                        
                        if len(elec_workers.loc[(elec_workers.naics_title == 'Utilities') & (elec_workers.occ_code == o), 'a_mean'].values) > 0:
                            to_append = pd.Series([tech, 
                                                   df.loc[df['naics'] == tech, 'naics_title'].values[0], \
                                                   o, 
                                                   df_full.loc[df_full['OCC_CODE'] == o, 'OCC_TITLE'].values[0], 
                                                   get_lvl(o), \
                                                   diff * e / sum_emp_full, # <- THIS IS IMPUTED TOT_EMP
                                                   elec_workers.loc[(elec_workers.naics_title == 'Utilities') & (elec_workers.occ_code == o), 'a_mean'].values[0], \
                                                   elec_workers.loc[(elec_workers.naics_title == 'Utilities') & (elec_workers.occ_code == o), 'a_median'].values[0], \
                                                   occ_up_prse,
                                                   occ_up],
                                                  index = df.columns)
                        else:
                            to_append = pd.Series([tech, 
                                                   df.loc[df['naics'] == tech, 'naics_title'].values[0], \
                                                   o, 
                                                   df_full.loc[df_full['OCC_CODE'] == o, 'OCC_TITLE'].values[0], 
                                                   get_lvl(o), \
                                                   diff * (e / sum_emp_full),  # <- THIS IS IMPUTED TOT_EMP
                                                   df_full.loc[df_full['OCC_CODE'] == o, 'A_MEAN'].values[0], \
                                                   df_full.loc[df_full['OCC_CODE'] == o, 'A_MEAN'].values[0], \
                                                   occ_up_prse,
                                                   occ_up],
                                                  index = df.columns)
                            
                        df = pd.concat([df.T, to_append], axis=1).T
                
                # Finally we remove any tiny occupations and rescale to fit
                sm_tot = df_tot.loc[(df_tot['naics'] == tech) & (df_tot['occ_group'] == 'total'), 'tot_emp'].sum()
                tiny = min(tiny_abs, tiny_frac * sm_tot)
                
                sm_occ = df.loc[(df['naics'] == tech) & (df['occ_code'].isin(new_occ_all)), 'tot_emp'].sum()
                df = df[(df['naics'] != tech) | (df['tot_emp'] > tiny)]
                sm_after = df.loc[(df['naics'] == tech) & (df['occ_code'].isin(new_occ_all)), 'tot_emp'].sum()
                if sm_after > 0:
                    df.loc[(df['naics'] == tech) & (df['occ_code'].isin(new_occ_all)), 'tot_emp'] *= \
                                                                        1 + (sm_occ - sm_after) / sm_after
                else:
                    print('imputed occs too small')
                    continue # continue without appending higher level becasue no occs are available

        elif diff > 0: # and/or lvl in [4,6]:
            # rescale existing
            scaling_factor = occ_up_emp / (occ_up_emp - diff)
            df.loc[(df['naics'] == tech) & (df['occ_code'].isin(occ_all)), 'tot_emp'] *= scaling_factor
            new_sm = df.loc[(df['naics'] == tech) & (df['occ_code'].isin(occ_all)), 'tot_emp'].sum()
        else:
            ; # nothing to do
            
        local_occ_set = [i for i in local_occ_set if i not in occ_oth]
        local_occ_set.append(occ_up)

In [ ]:
df.reset_index(inplace=True, drop=True)
df[['naics', 'occ_code', 'occ_up', 'emp_prse']].to_csv(data_out + 'bi-partite_emp_sum_elec_impute_parent_prse.csv')
elec_workers_ipums = df.drop('occ_up', axis=1).copy()

In [ ]:
# fill occs for which we have no wage data with wage data for that occupation in the entire utilities sector (221000)
for index, row in elec_workers_ipums[elec_workers_ipums.a_mean == '*'].iterrows():
    elec_workers_ipums.loc[index, 'a_mean'] = elec_workers_ipums[(elec_workers_ipums.naics == 221000) & (elec_workers_ipums.occ_code == row.occ_code)].a_mean.values[0]
    elec_workers_ipums.loc[index, 'a_median'] = elec_workers_ipums[(elec_workers_ipums.naics == 221000) & (elec_workers_ipums.occ_code == row.occ_code)].a_median.values[0]

In [ ]:
# utilities sector does not have wage for biological scientists or data entry keyers
elec_workers_ipums.loc[elec_workers_ipums.a_mean == '*']

In [ ]:
# We use national level data for these occuaptions
filename_occ_sect = "BLS/oesm18nat/national_M2018_dl.xlsx"
tab = "national_dl"
df_occ_wage = pd.read_excel(data + filename_occ_sect, sheet_name = tab, header = 0)
df_occ_wage = df_occ_wage[['OCC_CODE', 'A_MEAN', 'A_MEDIAN']].drop_duplicates()

In [ ]:
print(df_occ_wage.loc[df_occ_wage['OCC_CODE'] == '19-1020', :])
print(df_occ_wage.loc[df_occ_wage['OCC_CODE'] == '43-9021', :])

In [ ]:
# set final two items (in utilities) to the national average
elec_workers_ipums.loc[elec_workers_ipums['occ_code'] == '19-1020', 'a_mean'] = 86390
elec_workers_ipums.loc[elec_workers_ipums['occ_code'] == '43-9021', 'a_mean'] = 33740
elec_workers_ipums.loc[elec_workers_ipums['occ_code'] == '19-1020', 'a_mean'] = 77550
elec_workers_ipums.loc[elec_workers_ipums['occ_code'] == '43-9021', 'a_mean'] = 32170

In [ ]:
elec_names = elec_workers_ipums[['naics', 'naics_title']].drop_duplicates().copy()
elec_names = elec_names.set_index('naics')

In [ ]:
avg_wage = elec_workers_ipums[['naics', 'tot_emp', 'a_mean']].groupby('naics') \
                .apply(lambda x: np.average(x.a_mean, weights=x.tot_emp))
pd.concat([avg_wage, elec_names], axis=1).to_csv(data_out + 'elec_avg_wage.csv')

In [ ]:
# transmission_distribution = utilities - everything else
transmission_distribution = elec_workers_ipums[elec_workers_ipums.naics == 221100].copy()
for i, row in transmission_distribution.iterrows():
    oc = row['occ_code']
    empl = 0
    for tech in elec_tech:
        if len(elec_workers_ipums[(elec_workers_ipums['naics'] == int(tech)) & (elec_workers_ipums['occ_code'] == oc)]) > 0:
            empl += elec_workers_ipums.loc[(elec_workers_ipums['naics'] == int(tech)) & \
                                          (elec_workers_ipums['occ_code'] == oc), 'tot_emp'].values[0]
    transmission_distribution.loc[i, 'tot_emp'] -= empl

In [ ]:
elec_tech_tot_workers = elec_workers_ipums.loc[elec_workers_ipums.naics.isin([int(t) for t in elec_tech])].tot_emp.sum()
elec_gen_tot_workers = elec_workers_ipums[elec_workers_ipums.naics == 221100].tot_emp.sum()
trans_dist_tot_workers = transmission_distribution.tot_emp.sum()

transmission_distribution.tot_emp *= (elec_gen_tot_workers - elec_tech_tot_workers) / trans_dist_tot_workers

In [ ]:
transmission_distribution.naics = 221120
transmission_distribution.naics_title = 'Electricity Transmission and Distribution'

In [ ]:
elec_workers_ipums = pd.concat([elec_workers_ipums, transmission_distribution], axis=0)

In [ ]:
elec_workers_ipums[['naics', 'tot_emp']].groupby('naics').sum()

#### crosswalk

In [ ]:
# We use the BLS data. Remove the '_IPUMS' ending to do the rest of the analysis on ASEC codes
elec_workers_pivot = elec_workers_ipums.pivot_table(values='tot_emp', index='naics', columns='occ_code', fill_value=0).round(1)
elec_workers_pivot_IPUMS = elec_workers_pivot.dot(bls_ipums.loc[elec_workers_pivot.columns, :])

In [ ]:
#elec mean prse
elec_prse_mean = elec_workers_ipums[elec_workers_ipums.applymap(np.isreal).emp_prse].emp_prse.mean()

# impute mean in '**' cases
elec_workers_ipums.loc[elec_workers_ipums.applymap(np.isreal).emp_prse == False, 'emp_prse'] = elec_prse_mean

In [ ]:
elec_prse_pivot = elec_workers_ipums.pivot_table(values='emp_prse', index='naics', columns='occ_code', fill_value=0).round(1)

## merge with data

In [ ]:
emp_per_million_output = pd.read_csv(data_out + 'emp_per_million_output_sum_2018_no_elec_split.csv', index_col=0)

In [ ]:
emp_per_million_output.head()

In [ ]:
# BEA total output in 2018 of detailed electricty sectors, in millions of dollar
bea_electricity_output = pd.read_excel(data + "BEA/GrossOutput.xlsx", sheet_name='UGO305-A', usecols=[1, 24], skiprows=list(range(7)) + list(range(8, 29)), nrows=12)
bea_electricity_output.set_index('Industry', inplace=True)
eia_elec_output = pd.read_csv(data + 'US_generated_electricity.csv', index_col=1)

# BEA output only has 'fossil fuel', not coal and gas separately. We use
# EIA data with total GWh output to disaggregate the two, assuming relative monetary output
# reflects relative GWh output
gas_oil_frac_fossil = (eia_elec_output.loc[2018, 'oil'] + eia_elec_output.loc[2018, 'gas']) / \
    (eia_elec_output.loc[2018, 'oil'] + eia_elec_output.loc[2018, 'gas'] + eia_elec_output.loc[2018, 'coal'])

coal_frac_fossil = (eia_elec_output.loc[2018, 'coal']) / \
    (eia_elec_output.loc[2018, 'oil'] + eia_elec_output.loc[2018, 'gas'] + eia_elec_output.loc[2018, 'coal'])

bea_electricity_output.loc['Coal electric power generation', '2018'] = (coal_frac_fossil * \
                bea_electricity_output.loc['Fossil fuel electric power generation', '2018']).round()
bea_electricity_output.loc['Gas electric power generation', '2018'] = (gas_oil_frac_fossil * \
                bea_electricity_output.loc['Fossil fuel electric power generation', '2018']).round()
bea_electricity_output.drop('Fossil fuel electric power generation', axis=0, inplace=True)

In [ ]:
elec_workers_pivot['name'] = elec_workers_ipums[['naics', 'naics_title']].drop_duplicates().set_index('naics')
elec_workers_pivot = elec_workers_pivot.set_index('name')

elec_prse_pivot['name'] = elec_workers_ipums[['naics', 'naics_title']].drop_duplicates().set_index('naics')
elec_prse_pivot = elec_prse_pivot.set_index('name')

In [ ]:
# There is output data for more detailed industries (a, b), but occupational data only for
# the aggreagte industry (c = a + b). We split the number of workers accoring to the amount of total output
a = 'Electric Bulk Power Transmission and Control'
b = 'Electric Power Distribution'
c = 'Electricity Transmission and Distribution'

fact = bea_electricity_output.loc['Electric bulk power transmission and control', '2018'] / (bea_electricity_output.loc[\
                    'Electric bulk power transmission and control', '2018'] + \
                    bea_electricity_output.loc['Electric power distribution', '2018'])

elec_workers_pivot.loc[a, :] = elec_workers_pivot.loc[c, :] * fact
elec_workers_pivot.loc[b, :] = elec_workers_pivot.loc[c, :] * (1.0-fact)
# prse is a percentage. We take over the value
elec_prse_pivot.loc[a, :] = elec_prse_pivot.loc[c, :]
elec_prse_pivot.loc[b, :] = elec_prse_pivot.loc[c, :]

a = 'Gas Electric Power Generation'
b = 'Coal Electric Power Generation'
c = 'Fossil Fuel Electric Power Generation'

fact = bea_electricity_output.loc['Gas electric power generation', '2018'] / (bea_electricity_output.loc[\
                    'Gas electric power generation', '2018'] + \
                    bea_electricity_output.loc['Coal electric power generation', '2018'])

elec_workers_pivot.loc[a, :] = elec_workers_pivot.loc[c, :] * fact
elec_workers_pivot.loc[b, :] = elec_workers_pivot.loc[c, :] * (1.0-fact)
elec_prse_pivot.loc[a, :] = elec_prse_pivot.loc[c, :]
elec_prse_pivot.loc[b, :] = elec_prse_pivot.loc[c, :]

In [ ]:
elec_workers_pivot_select = elec_workers_pivot.iloc[2:].drop(['Electricity Transmission and Distribution', 
                                                              'Fossil Fuel Electric Power Generation'], axis=0)
elec_prse_pivot_select = elec_prse_pivot.iloc[2:].drop(['Electricity Transmission and Distribution', 
                                                        'Fossil Fuel Electric Power Generation'], axis=0)

In [ ]:
# variance is std_dev squared = ((prse * x_mean) / 100)^2
elec_var_pivot_select = (elec_workers_pivot_select.multiply(elec_prse_pivot_select) / 100)**2

In [ ]:
bipartite = pd.read_csv(data_out + 'bi-partite_emp_sum.csv', index_col = 0)

# fillna(0.0) because elec_workers_pivot does not contain all occupations (some are not employed in the utilities sector)
bipartite_elec = pd.concat([bipartite.iloc[:5], elec_workers_pivot_select.round(1), bipartite.iloc[6:]])
bipartite_elec.iloc[5:18, :] = bipartite_elec.iloc[5:18, :].fillna(value=0)

bipartite_elec.to_csv(data_out + 'bi-partite_emp_sum_elec.csv')

In [ ]:
bipartite = pd.read_csv(data_out + 'bi-partite_emp_sum_var.csv', index_col = 0)

# fillna(0.0) because elec_workers_pivot does not contain all occupations (some are not employed in the utilities sector)
bipartite_var = pd.concat([bipartite.iloc[:5], elec_var_pivot_select.round(1), bipartite.iloc[6:]])
bipartite_var.iloc[5:18, :] = bipartite_var.iloc[5:18, :].fillna(value=0)

bipartite_var.to_csv(data_out + 'bi-partite_emp_sum_var_elec.csv')

In [ ]:
# some data uses All Caps For Each Word, other data Only the first word
title_capitals = {'Biomass electric power generation': 'Biomass Electric Power Generation', 
                  'Electric bulk power transmission and control': 'Electric Bulk Power Transmission and Control',
                  'Electric power distribution': 'Electric Power Distribution',
                  'Geothermal electric power generation': 'Geothermal Electric Power Generation',
                  'Hydroelectric power generation': 'Hydroelectric Power Generation', 
                  'Natural gas distribution': 'Natural Gas Distribution', 
                  'Nuclear electric power generation': 'Nuclear Electric Power Generation',
                  'Other electric power generation': 'Other Electric Power Generation',
                  'Solar electric power generation': 'Solar Electric Power Generation',
                  'Water, sewage and other systems': 'Water, Sewage and Other Systems', 
                  'Wind electric power generation': 'Wind Electric Power Generation',
                  'Coal electric power generation': 'Coal Electric Power Generation',
                  'Gas electric power generation': 'Gas Electric Power Generation'}
title_lower = {v: k for k, v in title_capitals.items()}

In [ ]:
bea_electricity_output.index = bea_electricity_output.index.map(title_capitals)
bea_electricity_output = bea_electricity_output.loc[bea_electricity_output.index.dropna()]

In [ ]:
emp_per_million_output_elec = elec_workers_pivot.divide(bea_electricity_output['2018'], axis=0).dropna()
emp_per_million_output_elec = emp_per_million_output_elec.T

In [ ]:
emp_per_million_output_elec = pd.concat([emp_per_million_output.iloc[:, :5], emp_per_million_output_elec, emp_per_million_output.iloc[:, 6:]], axis=1)

In [ ]:
# Some occuaptions are not present in utilities, we fill them with 0
emp_per_million_output_elec.iloc[:, 5:18] = emp_per_million_output_elec.iloc[:, 5:18].fillna(0.0)

In [ ]:
emp_per_million_output_elec.to_csv(data_out + 'emp_per_million_output_sum_2018.csv')